In [15]:
import pandas as pd
import csv
import warnings
import openpyxl
warnings.filterwarnings("ignore")

In [17]:
from glob import glob

In [18]:
all_excel_files = glob("comments_reddit/*.xlsx")

df = pd.concat(pd.read_excel(excel_file,index_col=[0]) for excel_file in all_excel_files)

In [ ]:
#extract the english coments
from langdetect import detect, detect_langs
list_index=[]
for index,row in df.iterrows():
    try:
        language = detect(row['body'])
        if language=='en':
            list_index.append(index)
    except:
        pass

In [21]:
df_en=df[df.index.isin(list_index)]

In [27]:
texts = list(df_en['body'])

In [ ]:
#NER with Flair to extract books' names

from flair.models import SequenceTagger
from flair.data import Sentence

# Load the pre-trained NER tagger from Flair
tagger = SequenceTagger.load("ner")

# Function to process a batch of texts
def process_batch(texts):
    sentences = [Sentence(str(text)) for text in texts]  # Create Sentence objects
    tagger.predict(sentences)  # Predict NER tags for the batch

    # Extract 'MISC' entities
    batch_misc_entities = []
    for sentence in sentences:
        for entity in sentence.get_spans('ner'):
            if entity.tag == 'MISC' and 4 <= len(entity.text) <= 40:
                batch_misc_entities.append(entity.text)
    return batch_misc_entities

# Main processing loop with batching
def extract_misc_entities(texts, batch_size=500):
    misc_entities = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        misc_entities.extend(process_batch(batch_texts))
        if i % 10000 == 0:  # Progress logging
            print(f"Processed {i} records...")
    return misc_entities

# Use the function
misc_entities = extract_misc_entities(texts, batch_size=500)


In [28]:
#NER with Spacy to extract books' names

import spacy
nlp = spacy.load("en_core_web_sm")

def extract_work_of_art(texts):
    results = []
    for text in texts:
        doc = nlp(str(text))  # Process the text with spaCy
        work_of_art_entities = [ent.text for ent in doc.ents if ent.label_ == "WORK_OF_ART"]
        results.append(work_of_art_entities)
    return results


entities = extract_work_of_art(texts)